# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
df=('../output_data/cities.csv')
data=pd.read_csv(df)
data

,Unnamed: 0,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
0,0,Denver,39.7392,-104.9847,52.00,21,0,1.01,US,1616095856
1,1,Boston,42.3584,-71.0598,46.99,87,100,5.75,US,1616095937
2,2,Los Angeles,34.0522,-118.2437,68.00,52,1,5.41,US,1616096055
3,3,Oregon,44.0001,-120.5014,55.99,39,100,13.00,US,1616096228
4,4,Quito,-0.2299,-78.5249,64.40,63,75,8.05,EC,1616096187
...,...,...,...,...,...,...,...,...,...,...
566,583,Wukari,7.8500,9.7833,83.17,63,100,7.92,NG,1616140671
567,584,Sungai Penuh,-2.0833,101.3833,71.65,93,100,2.77,ID,1616140672
568,585,La Rinconada,37.4873,-5.9826,50.00,53,0,9.22,ES,1616140674
569,586,Moose Factory,51.2585,-80.6094,10.40,67,1,10.36,CA,1616140675


In [28]:
data.count()

Unnamed: 0    571
City          571
Latitude      571
Longitude     571
Max_temp      571
Humidity      571
Cloudiness    571
Wind_speed    571
Country       566
Date          571
dtype: int64

In [29]:
data.dropna(inplace = True) 
del data['Unnamed: 0']
data

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
0,Denver,39.7392,-104.9847,52.00,21,0,1.01,US,1616095856
1,Boston,42.3584,-71.0598,46.99,87,100,5.75,US,1616095937
2,Los Angeles,34.0522,-118.2437,68.00,52,1,5.41,US,1616096055
3,Oregon,44.0001,-120.5014,55.99,39,100,13.00,US,1616096228
4,Quito,-0.2299,-78.5249,64.40,63,75,8.05,EC,1616096187
...,...,...,...,...,...,...,...,...,...
566,Wukari,7.8500,9.7833,83.17,63,100,7.92,NG,1616140671
567,Sungai Penuh,-2.0833,101.3833,71.65,93,100,2.77,ID,1616140672
568,La Rinconada,37.4873,-5.9826,50.00,53,0,9.22,ES,1616140674
569,Moose Factory,51.2585,-80.6094,10.40,67,1,10.36,CA,1616140675


In [30]:
data.count()

City          566
Latitude      566
Longitude     566
Max_temp      566
Humidity      566
Cloudiness    566
Wind_speed    566
Country       566
Date          566
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
locations = data[['Latitude', 'Longitude']]
humid = data['Humidity']

In [63]:
fig = gmaps.figure()
layer=gmaps.heatmap_layer(locations, weights=humid,max_intensity=.5)
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# , dissipating = False, max_intensity = 75, point_radius = 1

In [68]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
tempa=data.loc[(data['Max_temp']>=65) & (data['Max_temp']<=90)]
tempa

humid=tempa.loc[(tempa['Humidity']>=5) & (tempa['Humidity']<=60)]
humid

windspeed=humid.loc[(humid['Wind_speed']>=5) & (humid['Wind_speed']<=30)]
windspeed

clouds_df=windspeed.loc[(windspeed['Cloudiness']>=5) & (windspeed['Cloudiness']<=80)]

clouds_df

# temp= data[data['Max_temp'] >=65 & data['Max_temp']<=90]

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
10,Hobart,-42.8794,147.3294,69.80,49,20,11.50,AU,1616139813
47,Mahabalipuram,12.6264,80.1722,87.80,55,40,8.05,IN,1616139884
73,Cape Town,-33.9258,18.4232,66.20,59,20,20.71,ZA,1616139684
177,Pindiga,9.9833,10.9333,84.36,52,75,12.30,NG,1616140079
186,Hong Kong,22.2855,114.1577,82.40,54,40,6.91,HK,1616140037
205,Pala,9.3642,14.9046,89.10,37,71,5.48,TD,1616140123
252,Yambio,4.5721,28.3955,88.65,42,50,5.64,SS,1616140197
261,Douentza,14.9951,-2.9517,82.22,8,51,5.82,ML,1616140212
317,Port Pirie,-33.1833,138.0167,82.40,44,67,12.66,AU,1616140301
332,Broken Hill,-31.9500,141.4333,80.60,41,40,20.71,AU,1616140323


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
# hotel_df=clouds_df

In [76]:
hotel_df=clouds_df.loc[:,:]
hotel_df

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
10,Hobart,-42.8794,147.3294,69.80,49,20,11.50,AU,1616139813
47,Mahabalipuram,12.6264,80.1722,87.80,55,40,8.05,IN,1616139884
73,Cape Town,-33.9258,18.4232,66.20,59,20,20.71,ZA,1616139684
177,Pindiga,9.9833,10.9333,84.36,52,75,12.30,NG,1616140079
186,Hong Kong,22.2855,114.1577,82.40,54,40,6.91,HK,1616140037
205,Pala,9.3642,14.9046,89.10,37,71,5.48,TD,1616140123
252,Yambio,4.5721,28.3955,88.65,42,50,5.64,SS,1616140197
261,Douentza,14.9951,-2.9517,82.22,8,51,5.82,ML,1616140212
317,Port Pirie,-33.1833,138.0167,82.40,44,67,12.66,AU,1616140301
332,Broken Hill,-31.9500,141.4333,80.60,41,40,20.71,AU,1616140323


In [109]:
# hotel_df['Hotel Name']=""
# hotel_df

In [ ]:
params= {"key": g_key,
        "radius": 5000,
         "types":"hotel",
        }

for index, row in hotel_df.iterrows():
    city=row['City']
    lat=row["Latitude"]
    lng=row['Longitude']
    
    params["location"]=f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address= requests.get(base_url,params=params)
    name_address=name_address.json()
    
#     print(json.dumps(name_address,indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index,"Hotel Name"]=name_address['results'][1]["name"]
#         clouds_df.loc[index,"Hotel Rating"]=name_address['results']["rating"]
    except(KeyError, IndexError):
        print("Missing field/results,,,skipping")
        time.sleep(1)
        

In [107]:
# hotel_df

In [108]:
# hotel_name=hotel_df['Hotel Name']
# hotel_name


In [41]:
# query_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel%20{city}&inputtype=textquery&&key={gkey}'

In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [106]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))